# Chat Models大语言模型

## 简介

在Agent智能体中，大语言模型提供理解、决策、调度等能力，是最基础的大脑模块。因此，本文我们介绍如何上手使用大语言模型。

简而言之，大语言模型基于深度学习技术，能够自动理解和生成人类自然语言文本的模型，广泛应用于文本生成、机器翻译、自动摘要、问答系统、情感分析等任务。

目前市面上已经有很多大语言模型，比如OpenAI的ChatGPT、百度的文心一言等等。大语言模型通常具有大量参数，对运行机器的显存和算量有很高的要求。为了方便大家使用，这些大语言模型都有对外提供相应的调用接口。

在`ERNIE Bot Agent`中，我们支持快速调用文心一言的多个模型，包括`ernie-3.5`、`ernie-turbo`、`ernie-4.0`和`ernie-longtext`。

| 模型名称 | 说明 | 功能 | 输入token数量上限 |
|:--- | :--- | :--- | :--- |
| ernie-3.5 | 文心大模型3.5版本。具备优秀的知识增强和内容生成能力，在文本创作、问答、推理和代码生成等方面表现出色。 | 对话补全，函数调用 | 3000 |
| ernie-turbo | 文心大模型。相比ernie-3.5模型具备更快的响应速度和学习能力，API调用成本更低。 | 对话补全 |  3000 |
| ernie-4.0 | 文心大模型4.0版本，具备目前系列模型中最优的理解和生成能力。 | 对话补全，函数调用 |  3000 |
| ernie-longtext | 文心大模型。在ernie-3.5模型的基础上增强了对长对话上下文的支持，输入token数量上限为7000。 | 对话补全，函数调用 |  7000 |

## 上手使用文心一言

### 安装

大家可以参考[官方文档](https://github.com/PaddlePaddle/ERNIE-Bot-SDK)，安装ERNIE Bot SDK。



### 鉴权

大家在使用ERNIE Bot Agent之前，需要完成鉴权步骤：

* 在[AI Studio星河社区](https://aistudio.baidu.com/index)注册并登录账号
* 在个人中心的[访问令牌页面](https://aistudio.baidu.com/index/accessToken)获取用户凭证`Access Token`
* 通过环境变量或者`Python`代码设置`Access Token`

In [8]:
# %env EB_AGENT_ACCESS_TOKEN=xxxxxx

import os

os.environ["EB_AGENT_ACCESS_TOKEN"] = "xxxxxx"

### 使用文心一言

首先，导入必要的依赖库。

In [9]:
import json
import asyncio
from erniebot_agent.chat_models import ERNIEBot
from erniebot_agent.memory import HumanMessage, AIMessage, SystemMessage, FunctionMessage

出于使用场景和性能的考虑，`ERNIE Bot Agent`只提供异步接口来调用文心一言模型。关于异步接口的详细介绍，请参考[asyncio文档](https://docs.python.org/3/library/asyncio.html)。

如下示例，我们首先创建文心一言`ernie-3.5`模型，然后两次调用`chat`接口传入只有单条`HumanMessage`的数组，文心一言模型会对单条`HumanMessage`做出回答，返回一条`AIMessage`。

In [10]:
async def demo_1():
    model = ERNIEBot(model="ernie-3.5")  # 创建模型
    human_message = HumanMessage(content="你好，你是谁")  # 定义输入信息
    ai_message = await model.chat(messages=[human_message])  # 调用模型chat接口，非流式返回
    print(ai_message.content, "\n")  # 输出结果

    human_message = HumanMessage(content="推荐三个深圳有名的景点")  # 定义输入信息
    ai_message = await model.chat(messages=[human_message], stream=True)  # 调用模型chat接口，流式返回
    async for chunk in ai_message:  # 流式输出结果
        print(chunk.content, end="")


await demo_1()

你好，我是文心一言，英文名是ERNIE Bot，可以协助你完成范围广泛的任务并提供有关各种主题的信息，比如回答问题，提供定义和解释及建议。如果你有任何问题，请随时向我提问。 

以下是三个深圳有名的景点推荐：

* **深圳湾公园**：位于深圳市福田区滨海大道，拥有美丽的海滨风光和丰富的绿色生态。这里是一个放松身心的理想之地，您可以欣赏到壮观的海景，同时还能在公园内散步、骑行和健身。
* **东部华侨城**：一个集文化、艺术、科技和娱乐于一体的综合性主题公园。这里拥有众多著名景点，如大侠谷、茶溪谷和云中部落等，每一个都充满了趣味和惊喜。
* **甘坑小镇**：位于深圳市龙岗区，是一个充满客家文化的小镇。这里保留了许多客家古建筑和传统文化，您可以在这里深入了解客家文化，同时品尝到独特的客家美食。

这些景点展现了深圳的自然风光、人文历史和现代气息，无论是欣赏海景、体验文化还是品尝美食，都能让您对深圳有更深刻的认识和感受。

如果希望文心一言模型能够根据多轮对话的上下文进行回答，我们需要将前面对话的输入输出`Message`带入后面对话，具体参考如下代码。

In [11]:
async def demo_2():
    model = ERNIEBot(model="ernie-3.5")
    messages = []  # 使用列表保存所有Message信息

    messages.append(HumanMessage(content="推荐三个深圳有名的景点"))
    ai_message = await model.chat(messages=messages)
    messages.append(ai_message)
    print(ai_message.content, "\n")

    messages.append(HumanMessage(content="根据你推荐的景点，帮我做一份一日游的攻略"))
    ai_message = await model.chat(messages=messages)
    messages.append(ai_message)
    print(ai_message.content, "\n")


await demo_2()

以下是深圳的三个有名的景点：

1. **深圳湾公园**：该公园位于深圳市福田区滨海大道，拥有美丽的海滨风光和丰富的绿色生态。这是一个放松身心的理想之地，您可以在这里欣赏壮观的海景，同时还能在公园内散步、骑行和健身。
2. **东部华侨城**：这是一个集文化、艺术、科技和娱乐于一体的综合性主题公园。这里拥有众多著名景点，如大侠谷、茶溪谷和云中部落等，每一个都充满了趣味和惊喜。
3. **仙湖植物园**：这是深圳市的一个著名植物园，拥有丰富的植物资源和优美的自然环境。这里有许多珍稀植物和特色花卉，同时还有许多文化景观和休闲设施，让您可以沉浸在自然之中，享受大自然的恩赐。

以上三个景点各具特色，无论是自然风光还是人文景观，都能让人流连忘返。 

好的，以下是一份基于你推荐的深圳景点的一日游攻略：

**深圳一日游攻略**

**早晨**：

* **早餐**：在深圳的繁华地区找一家当地特色的早餐店，品尝一下当地的早餐文化，比如肠粉、叉烧包等。

**上午**：

* **仙湖植物园**：首先前往仙湖植物园，欣赏丰富的植物资源，感受大自然的宁静与美丽。园内有各种珍稀植物和特色花卉，同时还有美丽的湖泊和园林景观。你可以悠闲地漫步其中，感受植物的生机与活力。

**中午**：

* **午餐**：在植物园附近的餐厅享用午餐，尝试一些当地的特色菜品。

**下午**：

* **东部华侨城**：午餐后，前往东部华侨城。这里是一个集多种娱乐元素于一体的主题公园，你可以选择自己喜欢的项目参与。例如，在大侠谷体验武侠世界的刺激，在茶溪谷感受清新的自然风光，在云中部落体验空中行走的惊险与刺激。这里不仅有丰富的娱乐项目，还有各种特色小吃和手工艺品，你可以在游玩的同时，品味当地的文化和美食。

**傍晚**：

* **深圳湾公园**：在夕阳西下的时候，来到深圳湾公园。在这里，你可以欣赏到美丽的海滨风光，同时感受滨海城市的浪漫与活力。你可以沿着海滨步道散步，或者骑行在海边的自行车道上，感受海风吹拂的惬意。在公园内还有一些健身设施和休闲区域，你可以在这里放松身心，享受一天的结束。

**晚上**：

* **晚餐**：在深圳湾公园附近的餐厅享用晚餐，可以选择海鲜或其他当地美食。餐后可以选择在市区内的一些夜市或购物中心逛逛，感受深圳的夜生活和购物文化。
* **返回住处**：最后返回酒店或住

下面示例，我们实现了一个简易的命令行聊天应用，可以和大语言模型网页端一样进行无限畅聊。

In [12]:
async def demo_3():
    model = ERNIEBot(model="ernie-3.5")
    messages = []

    print("你好，有什么我可以帮助你的吗？ (输入q可以退出聊天)")
    while True:
        prompt = input()
        if prompt == "q":
            break

        messages.append(HumanMessage(prompt))
        ai_message = await model.chat(messages=messages, stream=True)

        result = ""
        async for chunk in ai_message:
            result += chunk.content
            print(chunk.content, end="")
        print("")
        messages.append(AIMessage(result))


await demo_3()

你好，有什么我可以帮助你的吗？ (输入q可以退出聊天)


 广东省的省会是什么


广东省的省会是广州市。广州简称“穗”，别称羊城、花城，是广东省辖地级市、广东省省会、副省级市、国家中心城市、超大城市、广州都市圈核心城市，国务院批复确定的中国重要的中心城市、国际商贸中心和综合交通枢纽。


 广州市简称什么


广州市简称“穗”。


 q


下面示例，我们展示了文心一言模型如何使用FunctionCall。

In [13]:
# 定义function函数
def get_student_score(name: str) -> dict:
    info = {"小明": 80, "小红": 90, "小天": 95}
    if name in info:
        return {"score": info[name]}
    else:
        return f"we do not know the score of {name}"


# 定义function描述
functions = [
    {
        "name": "get_student_score",
        "description": "查询学生的分数",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "学生姓名",
                },
            },
            "required": [
                "name",
            ],
        },
        "responses": {
            "type": "object",
            "properties": {
                "score": {
                    "type": "integer",
                    "description": "分数",
                },
            },
        },
    },
]


async def demo():
    model = ERNIEBot(model="ernie-3.5")
    messages = []

    messages.append(HumanMessage("请问小明的分数是多少"))
    ai_message = await model.chat(messages=messages, functions=functions)  # 发送问题，带上function描述
    messages.append(ai_message)

    function_call = ai_message.function_call
    if function_call is not None:  # 如果返回的AIMessage有触发function，会有function_call字段
        name = function_call["name"]  # function_call的函数名称
        arguments = eval(function_call["arguments"])  # function_call的函数输入实参
        result = eval(name)(**arguments)  # 使用函数实参，调用并执行函数，拿到结果
        print("function_call:", function_call)
        print("function result:", result)

        function_message = FunctionMessage(
            name=name, content=json.dumps(result, ensure_ascii=False)
        )  # 构建FunctionMessage，封装函数的结果
        messages.append(function_message)

        ai_message = await model.chat(messages=messages, functions=functions)  # 将函数结果返回给模型，进行润色，得到最终输出
        print("final result:", ai_message.content)


await demo()

function_call: {'name': 'get_student_score', 'thoughts': '用户想要查询小明的分数，我需要使用查询学生分数的工具。', 'arguments': '{"name":"小明"}'}
function result: {'score': 80}
final result: 根据您的请求，小明的分数是80。如果您还有其他问题或需要了解更多信息，请随时告诉我。
